This tutorial will walk you through how to perform a regression task using supervised machine learning (ML) methods on different types of data that might be relevent for various scientific applications. First we must generate some representative data.

## Generating data

In [ ]:
# Activate working environment
using Pkg; Pkg.activate("../")
# using Pkg; Pkg.add("Plots")
# using Pkg; Pkg.add("MLUtils")
# using Pkg; Pkg.add("Flux")

In [ ]:
using Random 
using Plots
using Statistics
using Flux

First, we will generate some nonlinear multi-variate data that will serve as a sufficiently challenging test case for our ML model, we will use the Friedman #1 regression function which takes the form: $$f(x)=10sin(\pi x_1 x_2)+ 20(x_3−0.5)^2+10x_4 + 5x_5+ N(0,\sigma) $$

In the following distribution we will create output with 10 features despite the Friedman #1 function only depending upon the first 5 features. We will also add noise to the data to see how well the neural network models can fit perform.

In [ ]:
function make_friedman1(n_samples::Int64, n_features::Int64, noise::Float64, random_state::Int64)
    rng = Random.seed!(random_state)
    X = rand(rng, Float32, (n_features, n_samples))
    y = Array{Float32}(undef, (n_samples))
    @. y = 10sin(π*X[1, :]*X[2, :]) + 20(X[3, :]-0.5)^2 + 10X[4, :] + 5X[5, :] 
    y.+= noise.*randn(rng, Float32, (n_samples))
    return (X, reshape(y, 1, n_samples))
end

In [ ]:
# Create a non-linear distribution using the Friedman #1 function
n_samples = 1000
n_feats = 10
x, y = make_friedman1(n_samples, n_feats, 0.0, 42)
println("Data size: ", size(x), size(y))

In [ ]:
# Visualize the data using a mosaic plot
titles = permutedims(collect('A':'I'))
scatter(transpose(x[1:9, :]), transpose(y), markersize=2, layout=9, legend=false, title=titles)

## Preprocessing the data
Most ML problems optimally operate on data that is normalized. The optimal type of normalization depends upon the structure of the data and the desired output.

For this data, we will use a rather standard that shifts and scales the data to a distribution centered around 0 with a standard deviation equal to 1.

$$ x' = \frac{x - \mu(x)}{\sigma(x)}$$

We can do this manually, or use a number of pre-defined functions in sklearn. Here we will manually define the normalization functions, but in other tutorials we can explore different methods.

When creating a normalization function, it is often useful to also create an inverse normalization function which can be used to convert the ML model predictions back to the physical space.

In [ ]:
function normalize(X)
    μ = mean(X, dims=2)
    σ = stdm(X, μ, dims=2)
    X_normalized = (X.-μ)./σ
    return X_normalized, μ, σ
end

inverse_normalize(X_normalized, μ, σ) = X_normalized.*σ.+μ;

x_norm, x_mean, x_std = normalize(x);
y_norm, y_mean, y_std = normalize(y);

In [ ]:
# Visualize the normalized data 
scatter(transpose(x_norm[1:9, :]), transpose(y_norm), markersize=2, layout=9, legend=false, title=titles)

## Creating Train, Validation, and Test Datasets

We will now divide our dataset into three subdatasets for training, validation, and testing.
- Training Dataset: The dataset that we will use to optimize our ML model.
- Validation Dataset: A small, unseen dataset that we will use to monitor the performance of the ML model during training time. This can be useful for tuning hyperparameters and determining if the ML model is overfitting to the seen data.
- Test Dataset: Unseen data that is used to determine the overall performance of the fully-trained model.

Now we can generate training, validation and testing datasets with 80%-10%-10% split

In [ ]:
using MLUtils
train_data, val_data, test_data = splitobs((x_norm, y_norm); at=(0.8, 0.1) , shuffle=true);
println("Train data size: ", size(train_data[1]), size(train_data[2]))
println("Validation data size: ", size(val_data[1]), size(val_data[2]))
println("Test data size: ", size(test_data[1]), size(test_data[2]))

In [ ]:
# precompute the unnormalized versions of y
y_train_real = inverse_normalize(train_data[2], y_mean, y_std);
y_val_real = inverse_normalize(val_data[2], y_mean, y_std);
y_test_real = inverse_normalize(test_data[2], y_mean, y_std);

## Creating and training an ML Model
Now that the data is preprocessed nicely, let's create a standard feed-forward neural network and train it to learn the relationship between the features ($x$) and the output ($y$).

The number of hidden layers, neurons per layer, and a variety of other so-called "hyperparameters" will have a noticeable impact on model results. These values require extensive tuning depending on the problem at hand. For this example: 

- The initial model has an input layer, three hidden layers with number of neurons defined by the `width`, and an output layer, all layers with ReLU activation function.

- We also need to define the loss function, e.g. we can use the "mean square error" function (MSE) (which in `Flux` is named **`Flux.mse`**):
$$L(x, y) = \frac{1}{n}\sum_i^n \left[y_i - model(x_i) \right]^2, $$ 

- `Flux` also brings in lots of optimizers. For this example, we pick the `Adam` optimizer.

- Finally, to train the model, we will use batches of 32 samples.

In [ ]:
width = 8
model = Chain(
    Dense(n_feats => width, relu),   # activation function inside the layer
    Dense(width => width, relu),
    Dense(width => width, relu),
    Dense(width => 1))               # output layer with one feature

In [ ]:
learning_rate = 1e-3
batch_size = 32
# Define the loss function 
L(m, x, y) = Flux.mse(m(x), y)
# Define optimizer
optim = Flux.setup(Flux.Adam(learning_rate, (0.9, 0.8)), model); 
# Define the batch loader
loader = Flux.DataLoader(train_data, batchsize=batch_size, shuffle=false);

## Training the ML Model
Finally, we will train the ML model.

In [ ]:
num_epochs = 100

# Make a history arrays for storing metrics
train_loss = Float64[]
val_loss = Float64[]

# Training for 100 epoch and saving training and testing loss
@time for epoch in 1:num_epochs
    for batch in loader
        Flux.train!(L, model, [batch], optim)
    end
    push!(train_loss, L(model, train_data[1], train_data[2]))
    push!(val_loss, L(model, val_data[1], val_data[2]))
end

In [ ]:
plot(train_loss; lw=3, xaxis=("epochs"), yaxis="loss", label="train loss")
plot!(val_loss; lw=3, label="validation loss")

In [ ]:
println("Final train loss: $(train_loss[end])")
println("Final validation loss: $(val_loss[end])")

In [ ]:
# Evaluate the model on train and test data
pred_norm = model(x_norm);
pred_norm_train = model(train_data[1]);
pred_norm_val = model(val_data[1]);
pred_norm_test = model(test_data[1]);

# Unscale the results
pred_full = inverse_normalize(pred_norm, y_mean, y_std);
pred_train = inverse_normalize(pred_norm_train, y_mean, y_std);
pred_val = inverse_normalize(pred_norm_val, y_mean, y_std);
pred_test = inverse_normalize(pred_norm_test, y_mean, y_std);

In [ ]:
scatter(transpose(x[1:9, :]), transpose(y), markersize=2, layout=9, legend=false, title=titles)
scatter!(transpose(x[1:9, :]), transpose(pred_full), markersize=2, layout=9, legend=false)

## Compute some statistics on the fit

In [ ]:
function prediction_stats(pred, truth)
    rss = sum((pred .- truth).^2)
    tss = sum((truth .- mean(truth, dims=2)).^2)
    r_sq = 1 - rss/tss
    rmse = sqrt(Flux.mse(pred, truth))
    return r_sq, rmse
end

r_sq_train, rmse_train = prediction_stats(pred_train, y_train_real);
r_sq_val, rmse_val = prediction_stats(pred_val, y_val_real);
r_sq_test, rmse_test = prediction_stats(pred_test, y_test_real);

In [ ]:
p1 = scatter(transpose(y_train_real), transpose(pred_train), markershape=:star5, label="prediction", ylabel="prediction", xlabel="truth", title="Training Data")
plot!(p1, transpose(y_train_real), transpose(y_train_real), linestyle=:dash, label="x=y", color="grey")
annotate!(p1, 2.5, 21, Plots.text("R^2 = $(round(r_sq_train, digits=2)),\nRMSE = $(round(rmse_train, digits=2))", :left, 12))

p2 = scatter(transpose(y_val_real), transpose(pred_val), markershape=:star5, label="prediction", xlabel="truth", title="Validation Data")
plot!(p2, transpose(y_val_real), transpose(y_val_real), linestyle=:dash, label="x=y", color="grey")
annotate!(p2, 2.5, 21, Plots.text("R^2 = $(round(r_sq_val, digits=2)),\nRMSE = $(round(rmse_val, digits=2))", :left, 12))

p3 = scatter(transpose(y_test_real), transpose(pred_test), markershape=:star5, label="prediction", xlabel="truth", title="Test Data")
plot!(p3, transpose(y_test_real), transpose(y_test_real), linestyle=:dash, label="x=y", color="grey")
annotate!(p3, 2.5, 21, Plots.text("R^2 = $(round(r_sq_test, digits=2)),\nRMSE = $(round(rmse_test, digits=2))", :left, 12))

p = plot(p1, p2, p3, layout = (1, 3), link=:all,  size=(1200, 350))

## Exploration
Exploration of different hyperparameters can have a large impact on the training of the model. These are some examples of hyperparameters that can be examined in more detail:

- Hyperparameters to explore
- Model depth and width
- Layer regularization
- Layer initialization
- Learning rate
- Batch normalization
- Dropout layers
- Activation functions